In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
import seaborn as sns
%matplotlib inline

df = pd.read_csv('WELLCOME_APCspend2013_forThinkful.csv',encoding='Latin-1') #how we choose the encoding?
df.head()

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88


In [23]:
df.describe()

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
count,1928,2127,2126,2127,2127
unique,1880,299,984,2126,1402
top,Not yet available,Elsevier,PLoS One,"Exclusive breastfeeding, diarrhoel morbidity a...",£2040.00
freq,7,387,92,2,94


In [4]:


df['Journal title'] = df['Journal title'].replace(['PLOSONE', 'PLOS ONE', 'PLOS 1', 'PLOS'], 'PLOS ONE') 
df['Journal title'] = df['Journal title'].replace(['ACTA D', 'ACTA CRYSTALLOGRAPHICA SECTION D', 'ACTA CRYSTALLOGRAPHY D',
                                                                 'ACTA CRYSTALLOGRAPHICA, SECTION D', 'ACTA CRYSTALLOGRAPHICA SECTION D, BIOLOGICAL CRYSTALLOGRAPHY'],
                                                                'ACTA CRYSTALLOGRAPHICA SECTION D: BIOLOGICAL CRYSTALLOGRAPHY') 
df['Journal title'] = df['Journal title'].replace(['AMERICAN JNL EPIDEMIOLOGY'], 'AMERICAN JOURNAL OF EPIDEMIOLOGY') 
df['Journal title'] = df['Journal title'].replace(['AMERICAN JOURNAL OF MEDICAL GENETICS PART A'], 'AMERICAN JOURNAL OF MEDICAL GENETICS') 
df['Journal title'] = df['Journal title'].replace(['ANTIMICROBIAL AGENTS AND CHEMOTHERAPY', 'ANTIMICROBIAL AGFENTS AND CHEMOTHERAPY'], 'ANTIMICROBIAL AGENTS & CHEMOTHERAPY')
df['Journal title'] = df['Journal title'].replace(['ANGEWANDE CHEMIE', 'ANGEWANDTE CHEMIE INTERNATIONAL EDITION','ANGEW CHEMS INT ED' ], 'ANGEWANDTE CHEMIE')
df['Journal title']= df['Journal title'].replace(['BEHAVIOUR RESEARCH AND THERAPY'], 'BEHAVIOR RESEARCH & THERAPY') 
df['Journal title'] = df['Journal title'].replace(['BIOCHEM JOURNAL', 'BIOCHEMICAL JOURNALS'], 'BIOCHEMICAL JOURNAL') 
df['Journal title']= df['Journal title'].replace(['BIOCHEM SOC TRANS'], 'BIOCHEMICAL SOCIETY TRANSACTIONS') 
df['Journal title'] = df['Journal title'].replace(['BRITISH JOURNAL OF OPHTHALMOLOGY'], 'BRITISH JOURNAL OF OPTHALMOLOGY') 
df['Journal title'] = df['Journal title'].replace(['CELL DEATH DIFFERENTIATION'], 'CELL DEATH & DIFFERENTIATION') 
df['Journal title'] = df['Journal title'].replace(['CHILD: CARE, HEALTH DEVELOPMENT'], 'CHILD: CARE, HEALTH & DEVELOPMENT') 
df['Journal title'] = df['Journal title'].replace(['CURR BIOL'], 'CURRENT BIOLOGY')

From this table we can see that there are 199 PMID/PMCIDs(2127-1928) and 1 journal title(2127-2126) missing.From the readme file, we see that PMID/PMCIDs are only useful if we need to know whether an article is in PMC/Europe PMC which is not related to our analysis. So I wanna drop the entire column.

In [5]:
#Drops the PMID/PMCID column.
data = df.drop('PMID/PMCID',axis=1) 

The Readme file also acknowledges that we will see the same journal listed in different ways. So I wanna standardize them.

In [6]:
#Renames columns for ease of reference.
data.rename(columns={'Journal title': 'Journal', 'Article title':'Article', 'COST (£) charged to Wellcome (inc VAT when charged)':'Cost'}, inplace=True) 
#Drops any NA values  in any of the columns.
data = data.dropna(subset=['Publisher','Journal','Article'])

Because we gonna calculate the mean, median and standard deviation of Open-Access Cost per article for each journal. So we removes all currency symbol occurrences. 

In [7]:
data.Cost = data.Cost.map(lambda x: x.lstrip('£$').rstrip('£$'))

In [8]:
#Checks data type by series. 
data.dtypes

Publisher    object
Journal      object
Article      object
Cost         object
dtype: object

In [9]:
#Converts cost into a float: publisher, journal, and article are already strings.
data.Cost = data.Cost.map(lambda x: float(x))

We convert string columns to all-capital case to make them look cleaner.

In [10]:
data.Publisher = data.Publisher.str.upper()
data.Journal = data.Journal.str.upper()
data.Article = data.Article.str.upper()

Five Most Common Journals & Total Articles For Each:

In [11]:
common = data.groupby('Journal')[['Article']].count().sort_values(by=['Article'],ascending=False)
common.columns = ['Article Count']
common.head(5)

,Article Count
Journal,
PLOS ONE,192
JOURNAL OF BIOLOGICAL CHEMISTRY,53
NEUROIMAGE,29
PLOS GENETICS,24
PLOS PATHOGENS,24


Mean, Median, & Standard Deviation of Open-Access Cost Per Article for Each Journal

In [12]:
mean = data.groupby('Journal')[['Cost']].mean()
mean.columns = ['Mean']
meanperarticle = pd.concat([mean,common],axis=1)
meanperarticle.columns = ['Mean','Article Count']
meanperarticle['Mean Per Article'] = meanperarticle['Mean']/meanperarticle['Article Count']
meanperarticle.head(5)

C:\Users\mzz00\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  This is separate from the ipykernel package so we can avoid doing imports until


,Mean,Article Count,Mean Per Article
ACADEMY OF NUTRITION AND DIETETICS,2379.540,1,2379.5400
ACS CHEMICAL BIOLOGY,1418.186,5,283.6372
ACS CHEMICAL NEUROSCIENCE,1186.800,1,1186.8000
ACS NANO,668.140,2,334.0700
"ACTA CRYSTALLOGRAPHICA SECTION D, BIOLOGICAL CRYSTALLOGRAPHY",771.420,1,771.4200


In [13]:
mean = data.groupby('Journal')[['Cost']].mean()
mean.columns = ['Mean']
mean.head(5)

,Mean
Journal,
ACADEMY OF NUTRITION AND DIETETICS,2379.540
ACS CHEMICAL BIOLOGY,1418.186
ACS CHEMICAL NEUROSCIENCE,1186.800
ACS NANO,668.140
"ACTA CRYSTALLOGRAPHICA SECTION D, BIOLOGICAL CRYSTALLOGRAPHY",771.420


In [14]:
median = data.groupby('Journal')[['Cost']].median()
median.columns = ['Median']
median.head(5)

,Median
Journal,
ACADEMY OF NUTRITION AND DIETETICS,2379.54
ACS CHEMICAL BIOLOGY,1294.59
ACS CHEMICAL NEUROSCIENCE,1186.80
ACS NANO,668.14
"ACTA CRYSTALLOGRAPHICA SECTION D, BIOLOGICAL CRYSTALLOGRAPHY",771.42


In [15]:
#Should actually be dropping the NaN stdevs. Be careful what you impute...
deviation = data.groupby('Journal')[['Cost']].std()
deviation.columns = ['Standard Deviation']
deviation.head()

,Standard Deviation
Journal,
ACADEMY OF NUTRITION AND DIETETICS,NaN
ACS CHEMICAL BIOLOGY,507.309560
ACS CHEMICAL NEUROSCIENCE,NaN
ACS NANO,35.708892
"ACTA CRYSTALLOGRAPHICA SECTION D, BIOLOGICAL CRYSTALLOGRAPHY",NaN
